# Domain Adaptation using QLoRA

This notebook demonstrates how to:
1. Extract sentences from a technical PDF
2. Prepare MLM training data
3. Fine-tune a language model using QLoRA

In [1]:
!git clone https://github.com/arminwitte/mistral-peft mistralpeft

Cloning into 'mistralpeft'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 46 (delta 23), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 7.56 MiB | 24.89 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [2]:
import os
if not os.getcwd() == "/kaggle/working/mistralpeft":
    os.chdir("/kaggle/working/mistralpeft")
!pwd
!git pull 

/kaggle/working/mistralpeft
Already up to date.


In [3]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/

In [4]:
from mistralpeft.utils import TextExtractor, MLMPreprocessor, load_base_model, prepare_for_training, generate_response, CLAPreprocessor 
from transformers import Trainer, TrainingArguments, AutoTokenizer
from pathlib import Path

/usr/local/lib/python3.10/dist-packages/nltk/decorators.py:69: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  signature = inspect.formatargspec(regargs, varargs, varkwargs, defaults,


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface")

from huggingface_hub import login
login(secret_value_0) 


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1. Extract Sentences from PDF

First, we'll extract and clean sentences from the dissertation PDF.

In [6]:
# Initialize extractor
extractor = TextExtractor(language="en")  # or "de" for German

# Process the PDF
pdf_path = "Dissertation.pdf"
extraction_result = extractor.process_document(
    pdf_path,
    output_path="extracted_sentences.json"
)

print(f"Extracted {extraction_result['metadata']['num_sentences']} sentences")

# Preview some sentences
print("\nExample sentences:")
for sentence in extraction_result['sentences'][:3]:
    print(f"- {sentence[:512]}...")

Reading PDF: 100%|██████████| 315/315 [00:07<00:00, 39.70it/s]

Raw text length: 575529
Cleaned text length: 573488
Sentences: 1
Extracted 1 sentences

Example sentences:
- Technische Universität München Institut für Energietechnik Professur für Thermofluiddynamik Dynamics of Unsteady Heat Transfer and Skin Friction in Pulsating Flow Across a Cylinder Armin Witte Vollständiger Abdruck der von der Fakultät für Maschinenwesen der Technischen Universität München zur Erlangung des akademischen Grades eines DOKTOR – INGENIEURS genehmigten Dissertation. Vorsitzender: Prof. Dr.-Ing. Harald Klein Prüfer der Dissertation: 1. Prof. Wolfgang Polifke, Ph.D. 2. Prof. Dr.-Ing. Jens von Wolf...


## 2. Prepare MLM Training Data

Now we'll create masked language modeling examples for training.

In [7]:
model_name = "meta-llama/Llama-3.2-3B"  # Or the specific quantized version if you are using one.
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [8]:
json_file_paths = ["extracted_sentences.json"]
preprocessor = CLAPreprocessor(json_file_paths, tokenizer)
dataset = preprocessor.preprocess()

Token indices sequence length is longer than the specified maximum sequence length for this model (154693 > 131072). Running this sequence through the model will result in indexing errors


In [9]:
print(f"Created {len(dataset)} training and {len(dataset)} validation examples")

# Preview a training example
example = dataset[0]
print("\nExample input:")
print(preprocessor.tokenizer.decode(example['input_ids']))

Created 38 training and 38 validation examples

Example input:
<|begin_of_text|>Technische Universität München Institut für Energietechnik Professur für Thermofluiddynamik Dynamics of Unsteady Heat Transfer and Skin Friction in Pulsating Flow Across a Cylinder Armin Witte Vollständiger Abdruck der von der Fakultät für Maschinenwesen der Technischen Universität München zur Erlangung des akademischen Grades eines DOKTOR – INGENIEURS genehmigten Dissertation. Vorsitzender: Prof. Dr.-Ing. Harald Klein Prüfer der Dissertation: 1. Prof. Wolfgang Polifke, Ph.D. 2. Prof. Dr.-Ing. Jens von Wolfersdorf Die Dissertation wurde am 26.04.2018 bei der Technischen Universität München eingereicht und durch die Fakultät für Maschinenwesen am 09.10.2018 angenommen. Acknowledgments This thesis was conceived at the Thermo-Fluid Dynamics Group of the Technical University of Munich during my time as a research assistant. Financial support was provided by Deutsche Forschungsgemeinschaft (DFG), project PO 710/

## 3. Load and Prepare Model

We'll now load the base model and prepare it for QLoRA fine-tuning.

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [11]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Use load_in_8bit=True for 8-bit quantization
    )

base_model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto",
    quantization_config=quantization_config,
    )

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [12]:
from datasets import Dataset
train_test_set = dataset.train_test_split(test_size=0.1)

In [13]:
train_test_set 

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 34
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [14]:
# Load base model and tokenizer
#model, tokenizer = load_base_model()

# Prepare for LoRA training
model = prepare_for_training(
    base_model,
    lora_r=4,
    lora_alpha=16,
    lora_dropout=0.05
)

## 4. Train the Model

Now we'll fine-tune the model on our domain-specific data.

In [15]:
os.environ["WANDB_DISABLED"] = "true"

In [16]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="paged_adamw_8bit",
    log_level="debug"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_set['train'],
    eval_dataset=train_test_set['test']
)

# Start training

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using auto half precision backend
Currently training with a batch size of: 1
***** Running training *****
  Num examples = 34
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 102
  Number of trainable parameters = 2,293,760
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use

Epoch,Training Loss,Validation Loss
1,4.590700,4.223357
2,4.087300,3.792677
3,4.035300,3.663319


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)

***** Running Evaluation *****
  Num examples = 4
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-34
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "h

TrainOutput(global_step=102, training_loss=4.60345165051666, metrics={'train_runtime': 1840.8756, 'train_samples_per_second': 0.055, 'train_steps_per_second': 0.055, 'total_flos': 7071650549858304.0, 'train_loss': 4.60345165051666, 'epoch': 3.0})

## 5. Test the Model

Let's test the fine-tuned model with some domain-specific queries.

In [17]:
# Example queries about heat transfer and fluid dynamics
queries = [
    "Explain the relationship between pulsating crossflow and heat transfer efficiency.",
    "What are the key factors affecting skin friction in the experimental setup?",
    "Summarize the main findings regarding heat transfer dynamics in the study.",
    "How does the Reynolds number influence the observed phenomena?"
]

# Generate responses
for query in queries:
    print(f"\nQuery: {query}")
    response = generate_response(
        model,
        tokenizer,
        query,
        max_new_tokens=512,
        temperature=0.7
    )
    print(f"Response: {response}")
    print("-" * 80)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Query: Explain the relationship between pulsating crossflow and heat transfer efficiency.
Response: Explain the relationship between pulsatingflow and transfer efficiency
The is relation between puls cross and transfer efficiency is that puls flow a the high of cross to transfer heat transfer. puls flow transfer heat transfer in of to transfer. puls flow transfer heat transfer in the the the of of of of the. puls flow transfer heat transfer the the the the the. flow heat transfer heat transfer heat transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer transfer 

In [18]:
# Example queries about heat transfer and fluid dynamics
queries = [
    "Explain the relationship between pulsating crossflow and heat transfer efficiency.",
    "What are the key factors affecting skin friction in the experimental setup?",
    "Summarize the main findings regarding heat transfer dynamics in the study.",
    "How does the Reynolds number influence the observed phenomena?"
]

# Generate responses
for query in queries:
    print(f"\nQuery: {query}")
    response = generate_response(
        base_model,
        tokenizer,
        query,
        max_new_tokens=512,
        temperature=0.7
    )
    print(f"Response: {response}")
    print("-" * 80)


Query: Explain the relationship between pulsating crossflow and heat transfer efficiency.
Response: The relationship between pulsating flow and heat transfer is that puls flow can heat transfer in a. This flow is by in the of of flow. puls flow is and transfer. flow in. is puls. flow puls flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow flow

## 6. Save the Model

Finally, let's save our fine-tuned model for later use.

In [19]:
# Save the fine-tuned model
output_dir = Path("./final_model")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B/snapshots/13afe5124825b4f3751f836b40dafda64c1ed062/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "u

Model saved to final_model
